# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Develop-Thermodynamic-kinetic-Maximum-Entropy-Model" data-toc-modified-id="Develop-Thermodynamic-kinetic-Maximum-Entropy-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Develop Thermodynamic-kinetic Maximum Entropy Model</a></div><div class="lev2 toc-item"><a href="#Reactions-File" data-toc-modified-id="Reactions-File-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Reactions File</a></div><div class="lev3 toc-item"><a href="#Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix" data-toc-modified-id="Read-the-file-into-a-dataframe-and-create-a-stoichiometric-matrix-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Read the file into a dataframe and create a stoichiometric matrix</a></div><div class="lev2 toc-item"><a href="#Calculate-Standard-Free-Energies-of-Reaction" data-toc-modified-id="Calculate-Standard-Free-Energies-of-Reaction-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calculate Standard Free Energies of Reaction</a></div><div class="lev3 toc-item"><a href="#Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation" data-toc-modified-id="Output-the-Standard-Reaction-Free-Energies-for-use-in-a-Boltzmann-Simulation-122"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Output Standard Reaction Free Energies for Later Use</a></div><div class="lev2 toc-item"><a href="#Set-Fixed-Concentrations/Boundary-Conditions" data-toc-modified-id="Set-Fixed-Concentrations/Boundary-Conditions-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Set Fixed Concentrations/Boundary Conditions</a></div><div class="lev2 toc-item"><a href="#Prepare-model-for-optimization" data-toc-modified-id="Prepare-model-for-optimization-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Prepare model for optimization</a></div><div class="lev1 toc-item"><a href="#Nonlinear-Least-Squares-Optimization-of-Concentrations" data-toc-modified-id="Nonlinear-Least-Squares-Optimization-of-Concentrations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Nonlinear Least Squares Optimization of Concentrations</a></div><div class="lev2 toc-item"><a href="#Apply-Regulation-and-Optimize" data-toc-modified-id="Apply-Regulation-and-Optimize-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Apply MCA Regulation Method and Optimize</a></div><div class="lev1 toc-item"><a href="#ODE-Simulations" data-toc-modified-id="ODE-Simulations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ODE Simulations</a></div><div class="lev2 toc-item"><a href="#Calculate-Rate-Constants" data-toc-modified-id="Calculate-Rate-Constants-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Calculate Rate Constants</a></div><div class="lev2 toc-item"><a href="#ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE" data-toc-modified-id="ODE-Solvers:-Python-interface-using-libroadrunner-to-Sundials/CVODE-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Reinforcement Learning Based Regulation</a></div>

# Develop Thermodynamic-kinetic Maximum Entropy Model

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import subprocess
import sys
import re
import os
import warnings

from scipy.optimize import least_squares
from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

warnings.filterwarnings('ignore')

%matplotlib inline
T = 298.15
R = 8.314e-03
RT = R*T
N_avogadro = 6.022140857e+23
VolCell = 1.0e-15
Concentration2Count = N_avogadro * VolCell
concentration_increment = 1/(N_avogadro*VolCell)

use_experimental_data=True

In [46]:
1.48/4.2

0.35238095238095235

## Reactions File

In [11]:
with open( 'Gluconeogenesis.dat', 'r') as f:
    print(f.read())    

//
REACTION	 GLUCOSE_6_PHOSPHATASE
LEFT BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL
RIGHT BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO -17.057763138721384
DGZERO StdDev 0.7152372520174535
DGZERO-UNITS    KJ/MOL
//
REACTION	 PGI
LEFT D-FRUCTOSE_6-PHOSPHATE:CYTOSOL
RIGHT BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO 2.524005856552094
DGZERO StdDev 0.5967754160874962
DGZERO-UNITS    KJ/MOL
//
REACTION	 FBP
LEFT H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL
RIGHT D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DGZERO 0.0
DGZERO StdDev 0.0
DGZERO-UNITS    KJ/MOL
//
REACTION	 FBA
LEFT	GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL
RIGHT	D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL
LEFT_COMPARTMENT :CYTOSOL
RIGHT_COMPARTMENT :CYTOSOL
ENZYME_LEVEL 1.0
DG

### Read the file into a dataframe and create a stoichiometric matrix

In [12]:
fdat = open('Gluconeogenesis.dat', 'r')

left ='LEFT'
right = 'RIGHT'
left_compartment = 'LEFT_COMPARTMENT'
right_compartment = 'RIGHT_COMPARTMENT'
enzyme_level = 'ENZYME_LEVEL'
deltag0 = 'DGZERO'
deltag0_sigma = 'DGZERO StdDev'
same_compartment = 'Same Compartment?'
full_rxn = 'Full Rxn'

reactions = pd.DataFrame(index=[],columns=[left, right, left_compartment, right_compartment, enzyme_level, deltag0, deltag0_sigma, same_compartment,full_rxn])
reactions.index.name='REACTION'
S_matrix = pd.DataFrame(index=[],columns=[enzyme_level])
S_matrix.index.name='REACTION'

for line in fdat:
    if (line.startswith('REACTION')):
        rxn_name = line[9:-1].lstrip()
        S_matrix.loc[rxn_name,enzyme_level] = 1.0
        reactions.loc[rxn_name,enzyme_level] = 1.0
    if (re.match("^LEFT\s",line)):
        line = line.upper()
        left_rxn = line[4:-1].lstrip()
        left_rxn = re.sub(r'\s+$', '', left_rxn) #Remove trailing white space
        reactions.loc[rxn_name,left] = left_rxn
    elif (re.match('^RIGHT\s',line)):
        line = line.upper()
        right_rxn = line[5:-1].lstrip()
        right_rxn = re.sub(r'\s+$', '', right_rxn) #Remove trailing white space
        reactions.loc[rxn_name,right] = right_rxn    
    elif (line.startswith(left_compartment)):
        cpt_name = line[16:-1].lstrip()
        reactions.loc[rxn_name,left_compartment] = cpt_name
        reactants = re.split(' \+ ',left_rxn)
        for idx in reactants:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(-1.0);
                molecule = values[0]; 
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (line.startswith(right_compartment)):
        cpt_name = line[17:-1].lstrip()
        reactions.loc[rxn_name,right_compartment] = cpt_name
        products = re.split(' \+ ',right_rxn)
        for idx in products:
            values = re.split(' ', idx);
            if len(values) == 2:
                stoichiometry = np.float64(values[0]);
                molecule = values[1];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            else:
                stoichiometry = np.float64(1.0);
                molecule = values[0];
                if not re.search(':',molecule):
                    molecule = molecule + ':' + cpt_name
            S_matrix.loc[rxn_name,molecule] = stoichiometry;
    elif (re.match("^ENZYME_LEVEL\s", line)):
        level = line[12:-1].lstrip()
        reactions.loc[rxn_name,enzyme_level] = float(level)
        S_matrix.loc[rxn_name,enzyme_level] = float(level)       
    elif re.match('^COMMENT',line):
        continue
    elif re.match(r'//',line):
        continue
    elif re.match('^#',line):
        continue
        
fdat.close()
S_matrix.fillna(0,inplace=True)
S_active = S_matrix[S_matrix[enzyme_level] > 0.0]
active_reactions = reactions[reactions[enzyme_level] > 0.0]
del S_active[enzyme_level]

# Delete any columns/metabolites that have all zeros in the S matrix:
S_active = S_active.loc[:, (S_active != 0).any(axis=0)]
np.shape(S_active.values)
display(S_active.shape)
display(S_active)
reactions[full_rxn] = reactions[left] + ' = ' + reactions[right]

(11, 19)

,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL,ORTHOPHOSPHATE:CYTOSOL,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",GLYCERONE_PHOSPHATE:CYTOSOL,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,NADH:CYTOSOL,NAD+:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,ATP:CYTOSOL,ADP:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,PHOSPHOENOLPYRUVATE:CYTOSOL,OXALOACETATE:CYTOSOL,CO2:CYTOSOL,PYRUVATE:CYTOSOL
REACTION,,,,,,,,,,,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PGI,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBP,0.0,-1.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBA,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TPI,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GAPD,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PGK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0
PGM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
ENO,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0


In [4]:
for idx in reactions.index:
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    if re.search(':',boltzmann_rxn_str):
        all_cmprts = re.findall(':\S+', boltzmann_rxn_str)
        [s.replace(':', '') for s in all_cmprts] # remove all the ':'s 
        different_compartments = 0
        for cmpt in all_cmprts:
            if not re.match(all_cmprts[0],cmpt):
                different_compartments = 1
        if ((not different_compartments) and (reactions[left_compartment].isnull or reactions[right_compartment].isnull)):
            reactions.loc[idx,left_compartment] = cmpt
            reactions.loc[idx,right_compartment] = cmpt
            reactions.loc[idx,same_compartment] = True
        if different_compartments:
            reactions.loc[idx,same_compartment] = False
    else:
        if (reactions.loc[idx,left_compartment] == reactions.loc[idx,right_compartment]):
            reactions.loc[idx,same_compartment] = True
        else:
            reactions.loc[idx,same_compartment] = False
display(reactions)                        

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOS...
PGI,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL = BETA-D-GLUCOS...
FBP,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHAT...,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTO..."
FBA,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...
TPI,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL = GLYCERO...
GAPD,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...
PGK,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + ADP:...,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL = ...
PGM,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,2-PHOSPHO-D-GLYCERATE:CYTOSOL = 3-PHOSPHO-D-GL...
ENO,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,NaN,NaN,True,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL = 2-...


## Calculate Standard Free Energies of Reaction

The code below assumes that eQuilibrator is not installed as a package. The code requires eQuilibrator_api version 0.1.7 or 0.1.8. If you want to install this version of eQuilibrator_api, the required version is in 'eQuilibrator-api-v0.1.8'. Navigate to that folder and run 'python setup.py install' to build eQuilibrator v0.1.8 on your system. Or use ‘pip install "equilibrator_api==0.1.8".

In [5]:
import sys
sys.path.insert(0,'equilibrator-api-v0.1.8/build/lib')

from equilibrator_api import *
from equilibrator_api.reaction_matcher import ReactionMatcher
reaction_matcher = ReactionMatcher()
eq_api = ComponentContribution(pH=7.0, ionic_strength=0.25)  # loads data
for idx in reactions.index:
    print(idx, flush=True)
    boltzmann_rxn_str = reactions.loc[idx,'Full Rxn']
    full_rxn_str_no_cmprt = re.sub(':\S+','', boltzmann_rxn_str)
    print(full_rxn_str_no_cmprt)
    full_rxn_str_no_cmprt = re.sub('BETA-D-GLUCOSE','D-GLUCOSE',full_rxn_str_no_cmprt )
    rxn = reaction_matcher.match(full_rxn_str_no_cmprt)
    if not rxn.check_full_reaction_balancing():
        print('Reaction %s is not balanced:\n %s\n' % (idx, full_rxn_str_no_cmprt), flush=True)
    dG0_prime, dG0_uncertainty = eq_api.dG0_prime(rxn)
    display(dG0_prime, dG0_uncertainty)
    reactions.loc[idx,deltag0] = dG0_prime
    reactions.loc[idx,deltag0_sigma] = dG0_uncertainty

    reactions.loc['PYRt2m',deltag0] = -RT*np.log(10)
display(reactions)

GLUCOSE_6_PHOSPHATASE
BETA-D-GLUCOSE-6-PHOSPHATE + H2O = BETA-D-GLUCOSE + ORTHOPHOSPHATE


-8.998188834383427

0.6370893718226682

PGI
D-FRUCTOSE_6-PHOSPHATE = BETA-D-GLUCOSE-6-PHOSPHATE


-2.5220568042852847

...

7.639128086053029

Pyruvate_Carboxylase
ATP + PYRUVATE + CO2 + H2O = ADP + ORTHOPHOSPHATE + OXALOACETATE


-0.7958250266501636

7.604190202200161

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
REACTION,,,,,,,,,
GLUCOSE_6_PHOSPHATASE,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-8.998189,0.637089,True,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOS...
PGI,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-2.522057,0.596775,True,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL = BETA-D-GLUCOS...
FBP,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHAT...,:CYTOSOL,:CYTOSOL,1.0,-9.670922,0.818386,True,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTO..."
FBA,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",:CYTOSOL,:CYTOSOL,1.0,-21.45063,0.87227,True,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...
TPI,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-5.497984,0.753116,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL = GLYCERO...
GAPD,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,:CYTOSOL,:CYTOSOL,1.0,-5.242022,0.895659,True,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...
PGK,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + ADP:...,:CYTOSOL,:CYTOSOL,1.0,18.508339,0.889982,True,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL = ...
PGM,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-4.178737,0.65542,True,2-PHOSPHO-D-GLYCERATE:CYTOSOL = 3-PHOSPHO-D-GL...
ENO,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,4.0817,0.734193,True,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL = 2-...


### Output Standard Reaction Free Energies for Later Use

In [6]:
reaction_file = open('Gluconeogenesis.keq', 'w')
for y in reactions.index:
    print('%s\t%e' % (y, np.exp(-reactions.loc[y,'DGZERO']/RT)),file=reaction_file)
reaction_file.close()    

reaction_file.close()    
display((S_active.columns))

Index(['BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL', 'H2O:CYTOSOL',
       'BETA-D-GLUCOSE:CYTOSOL', 'ORTHOPHOSPHATE:CYTOSOL',
       'D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', 'D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL',
       'GLYCERONE_PHOSPHATE:CYTOSOL', 'D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL',
       '3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL', 'NADH:CYTOSOL',
       'NAD+:CYTOSOL', '3-PHOSPHO-D-GLYCERATE:CYTOSOL', 'ATP:CYTOSOL',
       'ADP:CYTOSOL', '2-PHOSPHO-D-GLYCERATE:CYTOSOL',
       'PHOSPHOENOLPYRUVATE:CYTOSOL', 'OXALOACETATE:CYTOSOL', 'CO2:CYTOSOL',
       'PYRUVATE:CYTOSOL'],
      dtype='object')

## Set Fixed Concentrations/Boundary Conditions

In [7]:
### Set Fixed Concentrations/Boundary Conditions
conc = 'Conc'
variable = 'Variable'
conc_exp = 'Conc_Experimental'
metabolites = pd.DataFrame(index = S_active.columns, columns=[conc,conc_exp,variable])
metabolites[conc] = 0.001
metabolites[variable] = True

metabolites.loc['2-PHOSPHO-D-GLYCERATE:CYTOSOL', conc] = 1.98e-01
metabolites.loc['PHOSPHOENOLPYRUVATE:CYTOSOL', conc] = 8.50e-02
metabolites.loc['OXALOACETATE:CYTOSOL', conc] = 1.000000e-03
metabolites.loc['3-PHOSPHO-D-GLYCERATE:CYTOSOL', conc] = 1.30e+01
metabolites.loc['3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL', conc] = 1.56e+00
metabolites.loc['GLYCERONE_PHOSPHATE:CYTOSOL', conc] = 4.18E-05
metabolites.loc['D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL', conc] = 7.41E-07
metabolites.loc['D-FRUCTOSE_6-PHOSPHATE:CYTOSOL', conc] = 3.18E-01
metabolites.loc['D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL', conc] = 9.60E-02
metabolites.loc['BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL', conc] = 3.80E-03

# Set the fixed metabolites:
metabolites.loc['PYRUVATE:CYTOSOL', conc] = 1.00E-03
metabolites.loc['PYRUVATE:CYTOSOL', variable] = False
metabolites.loc['BETA-D-GLUCOSE:CYTOSOL', conc] = 1.00E-11
metabolites.loc['BETA-D-GLUCOSE:CYTOSOL',variable] = False
metabolites.loc['ATP:CYTOSOL', conc] = 9.60E-03
metabolites.loc['ATP:CYTOSOL',variable] = False
metabolites.loc['ADP:CYTOSOL', conc] = 5.60E-04
metabolites.loc['ADP:CYTOSOL',variable] = False

# Low Ratio from NADP/NADPH
# NADP/NADPH = 2.1×10-6/ 1.2 ×10-4
# nadh/nad 2.10E-06 / 1.20E-04
metabolites.loc['NADH:CYTOSOL', conc] = 1.20E-04
metabolites.loc['NADH:CYTOSOL',variable] = False
metabolites.loc['NAD+:CYTOSOL', conc] = 2.10E-06
metabolites.loc['NAD+:CYTOSOL',variable] = False

metabolites.loc['ORTHOPHOSPHATE:CYTOSOL', conc] = 2.00E-06
metabolites.loc['ORTHOPHOSPHATE:CYTOSOL',variable] = False
metabolites.loc['CO2:CYTOSOL', conc] = 1.00E-04
metabolites.loc['CO2:CYTOSOL',variable] = False
metabolites.loc['H2O:CYTOSOL', conc] = 55
metabolites.loc['H2O:CYTOSOL',variable] = False    


#When loading experimental concentrations, first copy current 
#rule of thumb then overwrite with data values. Here we simply use rule of thumb concentrations.
metabolites[conc_exp] = metabolites[conc]

## Prepare model for optimization

- Adjust S Matrix to use only reactions with activity > 0, if necessary.
- Water stoichiometry in the stiochiometric matrix needs to be set to zero since water is held constant.
- The initial concentrations of the variable metabolites are random.
- All concentrations are changed to log counts.
- Equilibrium constants are calculated from standard free energies of reaction.
- R (reactant) and P (product) matrices are derived from S.

In [9]:
### Save a dictionary of the model that can be passed to optimization solvers

model_data = {}

#Stochiometric matrix

model_data['S'] = S_active.to_dict()

model_data['active_reactions'] = active_reactions.to_dict()

model_data['metabolites'] = metabolites.to_dict()


model_data['Keq'] = Keq_constant



In [1]:
import pickle

#with open('gluconeogenesis_prot4.pkl', 'wb') as handle:
#    pickle.dump(model_data, handle, protocol=4)

with open('gluconeogenesis_prot4.pkl', 'rb') as handle:
    b = pickle.load(handle)

#print(a == b)

In [15]:
import numpy as np
import pandas as pd
import subprocess
import sys
import re
import os
import warnings


import pickle



from scipy.optimize import least_squares
from IPython.core.display import display
from IPython.core.debugger import set_trace
pd.set_option('display.max_columns', None,'display.max_rows', None)

warnings.filterwarnings('ignore')

from importlib import reload



'''

Import Modules for optimizaion

'''

import flux_opt as Fopt




'''

########################################

Load in a model

###########################################

'''


model_file = 'gluconeogenesis_prot4.pkl'

#model_file = 'gluconeogenesis_DF.pkl'


with open(model_file, 'rb') as handle:
    model = pickle.load(handle)





Stoich_matrix = pd.DataFrame(model['S'])
active_reactions = pd.DataFrame(model['active_reactions'])
metabolites = pd.DataFrame(model['metabolites'])
Keq = pd.DataFrame({'Keq':model['Keq']}, index=Stoich_matrix.index)

In [8]:
Stoich_matrix

,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,OXALOACETATE:CYTOSOL,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",GLYCERONE_PHOSPHATE:CYTOSOL,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,PHOSPHOENOLPYRUVATE:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,ADP:CYTOSOL,CO2:CYTOSOL,NADH:CYTOSOL,ATP:CYTOSOL,NAD+:CYTOSOL,H2O:CYTOSOL,ORTHOPHOSPHATE:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL,PYRUVATE:CYTOSOL
GLUCOSE_6_PHOSPHATASE,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1.0,0.0
PGI,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
FBP,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,0.0,0.0
FBA,0.0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
TPI,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
GAPD,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0,1.0,0.0,0.0
PGK,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,0.0,0,0.0,0.0,0.0
PGM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
ENO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
PEP_Carboxykinase,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,-1.0,0.0,0,0.0,0.0,0.0


In [9]:
active_reactions

,LEFT,RIGHT,LEFT_COMPARTMENT,RIGHT_COMPARTMENT,ENZYME_LEVEL,DGZERO,DGZERO StdDev,Same Compartment?,Full Rxn
GLUCOSE_6_PHOSPHATASE,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOSOL,BETA-D-GLUCOSE:CYTOSOL + ORTHOPHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-8.998189,0.637089,True,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL + H2O:CYTOS...
PGI,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-2.522057,0.596775,True,D-FRUCTOSE_6-PHOSPHATE:CYTOSOL = BETA-D-GLUCOS...
FBP,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",D-FRUCTOSE_6-PHOSPHATE:CYTOSOL + ORTHOPHOSPHAT...,:CYTOSOL,:CYTOSOL,1.0,-9.670922,0.818386,True,"H2O:CYTOSOL + D-FRUCTOSE_1,6-BISPHOSPHATE:CYTO..."
FBA,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...,"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",:CYTOSOL,:CYTOSOL,1.0,-21.450630,0.872270,True,GLYCERONE_PHOSPHATE:CYTOSOL + D-GLYCERALDEHYDE...
TPI,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,GLYCERONE_PHOSPHATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-5.497984,0.753116,True,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL = GLYCERO...
GAPD,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...,D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL + ORTHOPH...,:CYTOSOL,:CYTOSOL,1.0,-5.242022,0.895659,True,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + NADH...
PGK,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL,3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL + ADP:...,:CYTOSOL,:CYTOSOL,1.0,18.508339,0.889982,True,3-PHOSPHO-D-GLYCERATE:CYTOSOL + ATP:CYTOSOL = ...
PGM,2-PHOSPHO-D-GLYCERATE:CYTOSOL,3-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,-4.178737,0.655420,True,2-PHOSPHO-D-GLYCERATE:CYTOSOL = 3-PHOSPHO-D-GL...
ENO,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL,2-PHOSPHO-D-GLYCERATE:CYTOSOL,:CYTOSOL,:CYTOSOL,1.0,4.081700,0.734193,True,PHOSPHOENOLPYRUVATE:CYTOSOL + H2O:CYTOSOL = 2-...
PEP_Carboxykinase,OXALOACETATE:CYTOSOL + ATP:CYTOSOL,CO2:CYTOSOL + ADP:CYTOSOL + PHOSPHOENOLPYRUVAT...,:CYTOSOL,:CYTOSOL,1.0,2.374871,7.639128,True,OXALOACETATE:CYTOSOL + ATP:CYTOSOL = CO2:CYTOS...


In [10]:
metabolites

,Conc,Conc_Experimental,Variable
BETA-D-GLUCOSE-6-PHOSPHATE:CYTOSOL,3.800000e-03,3.800000e-03,True
OXALOACETATE:CYTOSOL,1.000000e-03,1.000000e-03,True
D-FRUCTOSE_6-PHOSPHATE:CYTOSOL,3.180000e-01,3.180000e-01,True
"D-FRUCTOSE_1,6-BISPHOSPHATE:CYTOSOL",9.600000e-02,9.600000e-02,True
GLYCERONE_PHOSPHATE:CYTOSOL,4.180000e-05,4.180000e-05,True
D-GLYCERALDEHYDE-3-PHOSPHATE:CYTOSOL,7.410000e-07,7.410000e-07,True
3-PHOSPHO-D-GLYCEROYL_PHOSPHATE:CYTOSOL,1.560000e+00,1.560000e+00,True
PHOSPHOENOLPYRUVATE:CYTOSOL,8.500000e-02,8.500000e-02,True
2-PHOSPHO-D-GLYCERATE:CYTOSOL,1.980000e-01,1.980000e-01,True
3-PHOSPHO-D-GLYCERATE:CYTOSOL,1.300000e+01,1.300000e+01,True


In [16]:
Keq

,Keq
GLUCOSE_6_PHOSPHATASE,37.713965
PGI,2.766112
FBP,49.472797
FBA,5730.557425
TPI,9.188800
GAPD,8.287312
PGK,0.000572
PGM,5.396644
ENO,0.192698
PEP_Carboxykinase,0.383634


In [42]:
import simplesbml, cobra, libsbml
from cobra.io.sbml import F_REPLACE as f_replace

def pickle2simplesbml(Stoich_matrix: pd.DataFrame, reactions: pd.DataFrame, metabolites: pd.DataFrame, Keq: pd.DataFrame, objective_rxns: dict, compartment_volume)->simplesbml.SbmlModel:
    model = simplesbml.SbmlModel()
    compartments = set(met.split(':')[1] for met in metabolites.index)
    for compartment in compartments:
        model.addCompartment(compartment_volume[compartment], comp_id=compartment)
    for m in metabolites.index:
        if metabolites.loc[m,'Variable']:
            prefix = ''
        else:
            prefix='$'
        model.addSpecies(prefix + f_replace['F_SPECIE_REV'](m), metabolites.loc[m,'Conc'],comp=m.split(':')[1]) # needs to be in moles
    for rxn in reactions.index:
        left = [f_replace['F_SPECIE_REV'](specie) for specie in reactions.loc[rxn,'LEFT'].split(' + ')]
        right = [f_replace['F_SPECIE_REV'](specie) for specie in reactions.loc[rxn,'RIGHT'].split(' + ')]
        f = f"Keq*{'*'.join(left)}/({'*'.join(right)})"  # does not include nonunit stoichiometry
        model.addReaction(reactants= left, products=right,
                          expression = f"{f} - 1/{f}",
                         local_params={'Keq': Keq.loc[rxn,'Keq']},
                         rxn_id=f_replace['F_REACTION_REV'](rxn))
    sbml_ns = libsbml.SBMLNamespaces(3, 1)  # SBML L3V1
    sbml_ns.addPackageNamespace("fbc", 2)  # fbc-v2
    model.document.setNamespaces(sbml_ns.getNamespaces())

    model_fbc = model.model.getPlugin('fbc')
    objective = model_fbc.createObjective()
    objective.setId('OptBoltzmann')
    objective.setType('maximize')
    for rxn in objective_rxns:
        flux = objective.createFluxObjective()
        flux.setReaction(rxn)
        flux.setCoefficient(objective_rxns[rxn])
    model_fbc.setActiveObjective('OptBoltzmann')
    return model

In [28]:
def pickle2cobra(Stoich_matrix: pd.DataFrame, reactions: pd.DataFrame, metabolites: pd.DataFrame, Keq: pd.DataFrame, objective_rxns: dict, compartment_volume)->simplesbml.SbmlModel:
    model =cobra.Cobramodel

'M_fo__36__o'

In [43]:
pickle2simplesbml(Stoich_matrix, active_reactions, metabolites, Keq, {'FBP', 1}, {'CYTOSOL': 1e-12})

AttributeError: 'NoneType' object has no attribute 'createObjective'

In [6]:
'''

########################################

Specify the inputs for the optimization problem

###########################################

'''

'''

Some general notes on the expected data types and shapes
#########################

Stochiometric matrix: 
    - A matrix with dimension (number of reactions x number of metabolites )
    - Both forward and reverse reactions are captured with just one reaction row 
    - It is assumed metabolite columns are ordered such that all variable metabolites come first then all fixed metabolites are at the end


Metabolite upper bounds:
    - A vector of length equal to the number of variable metabolites that gives the upper bound for each
    - Note zero is the assumed lower bound for each metabolite


Fixed Metabolite log counts:
    - A vector of length equal to the number of fixed metabolites
    - Gives the value at which fixed metabolites will be set for the optimization

K :
    - The equilibrium constants for each reaction
    - A vector with length equal to number of reactions


Objective Coefficients:
    - A vector with length equal to the number of reactions giving the coefficient in the objective
    - assumption is objective is of the form <c,y> where c are the coefficients, y are fluxes, and <_,_> is the standard inner product

'''







'''
The Stochiometric matrix

'''

S = Stoich_matrix.values







'''
The fixed metabolite concentrations
'''


T = 298.15
R = 8.314e-03
RT = R*T
N_avogadro = 6.022140857e+23
VolCell = 1.0e-15
Concentration2Count = N_avogadro * VolCell
concentration_increment = 1/(N_avogadro*VolCell)


#number of variable metabolites
nvar = len(metabolites[ metabolites['Variable']==True].values)


fixed_concs = np.array(metabolites['Conc'].iloc[nvar:].values, dtype=np.float64)
fixed_counts = fixed_concs*Concentration2Count
f_log_counts = np.log(fixed_counts)







'''
The Variable Metabolite Upper bounds
'''

vcount_upper_bound = np.log(np.ones(nvar) *1.0e-03*Concentration2Count)





'''
Equilibrium Constants
'''


Keq = model['Keq']






'''
Objective Coefficients
'''


# Initial choice here as a single reaction we want max flux through
react_for_maxflux = 2


n_REACT = S.shape[0]
obj_coefs = np.zeros(n_REACT)
obj_coefs[react_for_maxflux] = 1






'''

Solve the Optimization problem with general method

'''


y_sol, alpha_sol, n_sol = Fopt.flux_ent_opt(obj_coefs,vcount_upper_bound,f_log_counts,S,Keq)

Ipopt 3.14.7: max_iter=10000
max_cpu_time=800000
tol=1e-07
acceptable_tol=1e-06
linear_solver=mumps


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.7, running with linear solver MUMPS 5.4.0.

Number of nonzeros in equality constraint Jacobian...:       98
Number of nonzeros in inequality constraint Jacobian.:      108
Number of nonzeros in Lagrangian Hessian.............:       22

Total number of variables............................:       55
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       11
                     variables with only upper bounds:  

In [3]:
y_sol

array([0.64710692, 0.64710692, 0.64710692, 0.64710692, 0.64710692,
       1.29421384, 1.29421384, 1.29421384, 1.29421384, 1.29421384,
       1.29421384])

In [4]:
alpha_sol

array([1.00000305, 1.00000296, 1.00000306, 1.00000321, 1.000003  ,
       1.00000179, 1.00000161, 1.00000178, 1.00000172, 1.00000173,
       0.00109527])

In [5]:
n_sol

array([-1.33042504, 13.30836837, -2.02971018,  1.48078474, -2.47739902,
       -4.37722669, -2.83495049,  3.57730507,  1.3218781 ,  2.39885926])